
**[PT]** Português

---

**[EN]** English



# Visão geral da base de dados

---

# Database overview 


## Setup

In [1]:
from timelinknb import get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)

## Database status

In [2]:
from sqlalchemy import select,text
from timelinknb import Session

with Session() as session:
    classes = session.execute(
        select(
            text('class,count(*) as n from entities group by class')
        )
    )
    for c,n in classes:
        print(f'{n:8} | {c}')


     235 | act
 2850169 | attribute
      13 | class
  155307 | person
  205409 | relation
     235 | source


## Atributos inferidos

---

## Atributes inferred

In [3]:
from sqlalchemy import func
from sqlalchemy import select
from timelinknb import get_attribute_table

attr_table = get_attribute_table()

# nml = %sql select the_type, count(*) as tot from attributes group by the_type
stmt = select(attr_table.c.the_type,func.count().label('tot')).group_by('the_type')
print(stmt)
print()
attributes_extracted = []
with Session() as session:
    nml = session.execute(stmt)
    for the_type, tot in nml:
        print(f'{tot:6} | {the_type}')
        attributes_extracted.append((the_type,tot))

attr_not_redundand = [(t,c) for (t,c) in attributes_extracted if "." not in t]

SELECT attributes.the_type, count(*) AS tot 
FROM attributes GROUP BY attributes.the_type

   356 | colegio
105298 | código-de-referência
105298 | data-do-registo
    16 | errata
 53697 | exame
104638 | faculdade
  7167 | faculdade-original
104638 | faculdade.ano
 87112 | grau
 87112 | grau.ano
 40672 | instituta
 40672 | instituta.ano
  2444 | matricula-classe
  2444 | matricula-classe.ano
   114 | matricula-classe.obrigado
   114 | matricula-classe.obrigado.ano
    32 | matricula-classe.ordinário
    32 | matricula-classe.ordinário.ano
     2 | matricula-classe.voluntário
     2 | matricula-classe.voluntário.ano
   170 | matricula-curso
   170 | matricula-curso.ano
313032 | matricula-faculdade
313032 | matricula-faculdade.ano
  9475 | matricula-faculdade.obrigado
  9475 | matricula-faculdade.obrigado.ano
  6512 | matricula-faculdade.ordinário
  6512 | matricula-faculdade.ordinário.ano
  1546 | matricula-faculdade.voluntário
  1546 | matricula-faculdade.voluntário.ano
  1373 | matricu

### Atributos inferidos (sem os atributos redundantes)

Os atributos redundantes são repetições 
de um atributos principal para facilitar 
determinadas pesquisas.

Todos os atributos com um "." no nome fornecem
uma variante do nome ou dovalor para facilitar pesquisas
e contagens. 

Por exemplo:

---

### Inferred attributes (without redundant attributes)

Redundant attributes are repetitions of an important
attribute with variants of the name or value to
facilitate retrieval or counts.

For instance:



| Atributo                     | Valor        | data 
|------------------------------|--------------|-----------
| faculdade                    | Cânones      | 1774-11-15
| faculdade.ano  	           | Cânones.1774 | 1774-11-15
| matricula-faculdade          | Matemática   |1775-00-00
| matricula-faculdade.obrigado | Matemática   | 1775-00-00



In [4]:
import pandas as pd

df = pd.DataFrame.from_records(attr_not_redundand, columns=['attr','n'])
df.sort_values('attr')

,attr,n
0,colegio,356
1,código-de-referência,105298
2,data-do-registo,105298
3,errata,16
4,exame,53697
5,faculdade,104638
6,faculdade-original,7167
7,grau,87112
8,instituta,40672
9,matricula-classe,2444


### Exemplo de registo complexo

---

### Example of complex record

Example of the current capabilities of the algorithm including correction of “Faculdade”, religious order, titles, enrollment at class level, exam results and degrees.

In [5]:
from timelinknb.pandas import group_attributes
from ucalumni.extractors import get_extractors
from ucalumni.aluno import get_and_process_aluno

id = '316297'

exts = get_extractors()
print("Extractors currently active:")
[print("  ",ex.__name__) for ex in exts]
print()

print()
print("Copy the information to a markdown file it should render nicely")
print("------------------------")
aluno = get_and_process_aluno(id,db)
print(aluno)
print("------------------------")


Extractors currently active:
   extract_name_note_vid
   extract_colegio
   extract_ordem_religiosa
   extract_naturalidade
   extract_instituta
   extract_faculdade
   extract_graus
   extract_matriculas
   extract_exames


Copy the information to a markdown file it should render nicely
------------------------
# 316297 João Pedro Ribeiro

## Original
```
Id: 316297
Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/R/005692

Nome        : João Pedro Ribeiro
Data inicial: 1774-11-15
Data final  : 1781-05-06
Filiação: Pedro do Rosário Ribeiro

Naturalidade: Porto
Faculdade: Direito - Cânones

Matrículas: 1.º ano jurídico 15.11.1774, Vol. III, L. I, fl. 20v.
2.º jurídico 1775, Vol. IV, L. I, fl. 48
1.º ano matemática (obrigado) 1775, Vol. IV, L. 4, fl. 48
2.º ano jurídico 18.10.1776, Vol. V, L. I, fl. 35v.
1.º ano matemática (obrigado) 08.10.1776, Vol. V, L. IV, fl. 12v.
3.º ano cânones 06.10.1777, Vol. VI, fl. 10
1.º ano filosofia 16.10.1777, Vol. VI, L. VI, fl. 13
4.º ano cânones 02.10

#### Mostrar os attributos como uma tabela

---

#### Show the attributes as a table

In [6]:

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_colwidth',100)

df = group_attributes([id])
df = df[~df['type'].str.contains(".", regex = False)]  # remove redundant attributes with "." in the name
df = df[['date','type','value','attr_obs']].sort_values(['date','type','value'])
df.fillna(" ")

,date,type,value,attr_obs
id,,,,
316297,1774-11-15,faculdade,Cânones,Faculdade corrigida
316297,1774-11-15,faculdade-original,Direito - Cânones,
316297,1774-11-15,matricula-classe,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,1774-11-15,matricula-curso,Curso jurídico,"Vol. III, L. I, fl. 20v."
316297,1774-11-15,naturalidade,Porto,
316297,1774-11-15,nome,João Pedro Ribeiro,
316297,1774-11-15,nome-apelido,Pedro Ribeiro,
316297,1774-11-15,nome-apelido,Ribeiro,
316297,1774-11-15,nome-geografico,Porto,
